In [2]:
pip install category_encoders xgboost lightgbm catboost scorecardpy optuna mlxtend xlrd==2.0.1


     ---------------------------------------- 0.0/58.1 kB ? eta -:--:--
     ------- -------------------------------- 10.2/58.1 kB ? eta -:--:--
     ------------- ------------------------ 20.5/58.1 kB 330.3 kB/s eta 0:00:01
     -------------------------- ----------- 41.0/58.1 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 58.1/58.1 kB 339.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for scorecardpy: filename=scorecardpy-0.1.9.7-py3-none-any.whl size=60667 sha256=4439ebfb1329bc5d53de04d821bd0bb0add0cdc44d3865097d181cc1d2d59464
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\68\88\65\58d6aa058698c47d7a24352952ea07a1409cc2eff3a8087c0c
Successfully built scorecardpy
Note: you may need to restart the kernel to use updated packages.


In [146]:
import pandas as pd  # Import pandas for data manipulation and analysis
import numpy as np  # Import numpy for numerical operations
from sklearn.model_selection import train_test_split  # Import train_test_split for splitting data into training and testing sets
from sklearn.preprocessing import StandardScaler, LabelEncoder  # Import StandardScaler for feature scaling and LabelEncoder for encoding target labels
from sklearn.linear_model import LogisticRegression  # Import LogisticRegression for logistic regression model
from sklearn.ensemble import RandomForestClassifier, StackingClassifier  # Import RandomForestClassifier for random forest model and StackingClassifier for model stacking
from sklearn.svm import SVC  # Import SVC for support vector classifier
from xgboost import XGBClassifier  # Import XGBClassifier for XGBoost model
from lightgbm import LGBMClassifier  # Import LGBMClassifier for LightGBM model
from catboost import CatBoostClassifier  # Import CatBoostClassifier for CatBoost model
from sklearn import metrics  # Import metrics from sklearn for model evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_auc_score  # Import specific metrics for model evaluation
import warnings  # Import warnings to manage warnings
import category_encoders as ce  # Import category_encoders for encoding categorical features
import optuna
# Ignore warnings
warnings.filterwarnings('ignore')  # Suppress warnings for cleaner output

In [119]:
path = r'C:\Users\User\Desktop\Rashad\DATA\default of credit card clients.xls'

# Load dataset from Excel file
data = pd.read_excel(path)

# Set option to display all columns
pd.set_option('display.max_columns', None)

# Display the loaded dataset
data

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,1,20000,female,university,married,24,2,2,-1,-1,-2,-2,3913,3102,689,0,0,0,0,689,0,0,0,0,1
1,2,120000,female,university,single,26,-1,2,0,0,0,2,2682,1725,2682,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,female,university,single,34,0,0,0,0,0,0,29239,14027,13559,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,female,university,married,37,0,0,0,0,0,0,46990,48233,49291,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,male,university,married,57,-1,0,-1,0,0,0,8617,5670,35835,20940,19146,19131,2000,36681,10000,9000,689,679,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000,male,high school,married,39,0,0,0,0,0,0,188948,192815,208365,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,male,high school,single,43,-1,-1,-1,-1,0,0,1683,1828,3502,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,male,university,single,37,4,3,2,-1,0,0,3565,3356,2758,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,male,high school,married,41,1,-1,0,0,0,-1,-1645,78379,76304,52774,11855,48944,85900,3409,1178,1926,52964,1804,1


In [120]:
df.dtypes

ID            int64
LIMIT_BAL     int64
SEX          object
EDUCATION    object
MARRIAGE     object
AGE           int64
PAY_0         int64
PAY_2         int64
PAY_3         int64
PAY_4         int64
PAY_5         int64
PAY_6         int64
BILL_AMT1     int64
BILL_AMT2     int64
BILL_AMT3     int64
BILL_AMT4     int64
BILL_AMT5     int64
BILL_AMT6     int64
PAY_AMT1      int64
PAY_AMT2      int64
PAY_AMT3      int64
PAY_AMT4      int64
PAY_AMT5      int64
PAY_AMT6      int64
default       int64
dtype: object

# Pre-processing

In [127]:
models = ["logistic_regression", "random_forest", "svc", "xgboost", "lightgbm", "catboost", "catboost_custom"]

# Create copies of the DataFrame for different preprocessing approaches
df_copy = df.copy()
df_boost = df.copy()
df_cat = df.copy()

# Iterate through each model in the list
for model in models:
    if model == "logistic_regression" or model == "svc":
        # Preprocessing for logistic regression and SVC models
        
        # Fill missing values with mean for numerical columns and mode for categorical columns
        for col in df_copy.columns:
            if df_copy[col].dtype in ['int64', 'float64']:
                df_copy[col] = df_copy[col].fillna(df_copy[col].mean())
            elif df_copy[col].dtype == 'object':
                df_copy[col] = df_copy[col].fillna(df_copy[col].mode().iloc[0])
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_copy.columns.tolist()
        all_columns.remove('default')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_copy[all_columns] = encoder.fit_transform(df_copy[all_columns], df_copy['default'])
        
        # Remove duplicate rows and 'ID' column if exists
        df_copy.drop_duplicates(inplace=True)
        if 'ID' in df_copy.columns:
            df_copy.drop(columns=['ID'], inplace=True)
                
    elif model in ["random_forest", "xgboost", "lightgbm", "catboost"]:
        # Preprocessing for tree-based models
        
        # Perform Weight of Evidence Encoding on all columns except the target column 'default'
        all_columns = df_boost.columns.tolist()
        all_columns.remove('default')
        encoder = ce.WOEEncoder(cols=all_columns)
        df_boost[all_columns] = encoder.fit_transform(df_boost[all_columns], df_boost['default'])
             
    elif model ==  "catboost_custom":
        # Preprocessing for CatBoost with custom handling of missing values
        
        # Fill missing values in specified categorical columns with 'Missing Value'
        numerical_cols = df_cat.select_dtypes(include=['int64', 'float64']).columns.tolist()
        numerical_cols.remove('default')
        columns_to_fill = cat_features=['SEX', 'EDUCATION', 'MARRIAGE']
        df_cat[columns_to_fill] = df_cat[columns_to_fill].fillna('Missing Value')
        
        # Perform Weight of Evidence Encoding on numerical columns
        encoder = ce.WOEEncoder(cols=numerical_cols)
        df_cat[numerical_cols] = encoder.fit_transform(df_cat[numerical_cols], df_cat['default'])

# Split data into features (X) and target (y) for each preprocessing approach
x_log = df_copy.drop(columns=['default'])
y_log = df_copy['default']
X_train_log, X_test_log, y_train_log, y_test_log = train_test_split(x_log, y_log, test_size=0.3, random_state=42)

x_boost = df_boost.drop(columns=['default'])
y_boost = df_boost['default']
X_train_boost, X_test_boost, y_train_boost, y_test_boost = train_test_split(x_boost, y_boost, test_size=0.3, random_state=42)

x_cat = df_cat.drop(columns=['default'])
y_cat = df_cat['default']
X_train_cat, X_test_cat, y_train_cat, y_test_cat = train_test_split(x_cat, y_cat, test_size=0.3, random_state=42)

In [128]:
models = []  # Initialize an empty list to store model definitions

# Define default models for each algorithm
xgb_model_def = XGBClassifier()
lgb_model_def = LGBMClassifier()
catboost_model_def = CatBoostClassifier()
catboost_model_custom = CatBoostClassifier(cat_features=['SEX', 'EDUCATION', 'MARRIAGE'])
lg = LogisticRegression()
rf = RandomForestClassifier()
svc_model_def = SVC(probability=True)  # Add SVC model definition

# Define models for stacking
stacking_models = [('XGBoost', xgb_model_def),
                   ('LightGBM', lgb_model_def),
                   ('CatBoost', catboost_model_def),
                   ('CatBoost_Custom', catboost_model_custom),
                   ('LogisticRegression', lg),
                   ('RandomForest', rf),
                   ('SVC', svc_model_def)]  # Add SVC to stacking_models

# Extend models list with default models
models.extend([
    ('XGBoost', xgb_model_def),
    ('LightGBM', lgb_model_def),
    ('CatBoost', catboost_model_def),
    ('CatBoost_Custom', catboost_model_custom),
    ('LogisticRegression', lg),
    ('RandomForest', rf),
    ('SVC', svc_model_def)  # Add SVC to models list
])

In [129]:
def train_and_evaluate_model(model_name, model, X_train, y_train, X_test, y_test):
    """
    Train and evaluate the given model on the training and testing data.

    Parameters:
    model_name (str): Name of the model for display purposes.
    model : Machine learning model object.
    X_train : Features of the training data.
    y_train : Target labels of the training data.
    X_test : Features of the testing data.
    y_test : Target labels of the testing data.

    Returns:
    float: Gini coefficient calculated from the model's predictions.
    """

    # Fit the model on the training data
    model.fit(X_train, y_train)

    # Predict labels and probabilities on the testing data
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    # Calculate ROC AUC and Gini coefficient
    roc_prob = roc_auc_score(y_test, y_prob)
    gini_prob = roc_prob * 2 - 1

    # Calculate confusion matrix and classification report
    confusion_matrix_result = confusion_matrix(y_test, y_pred)
    classification_report_result = classification_report(y_test, y_pred)

    # Print model performance metrics
    print(f'Model Performance for {model_name}')
    print('Gini prob is', gini_prob * 100)
    print(classification_report_result)
    print(confusion_matrix_result)

    return gini_prob

In [130]:
gini_df = pd.DataFrame(columns=['Model', 'Gini_prob'])

# Iterate through each model in the list of models
for model_name, model in models:
    # Train and evaluate the model, and calculate the Gini coefficient
    if model_name == 'CatBoost_Custom':
        gini_prob = train_and_evaluate_model(model_name, model, X_train_cat, y_train_cat, X_test_cat, y_test_cat)
    elif model_name in ['LogisticRegression', 'SVC']:
        gini_prob = train_and_evaluate_model(model_name, model, X_train_log, y_train_log, X_test_log, y_test_log)
    else:
        gini_prob = train_and_evaluate_model(model_name, model, X_train_boost, y_train_boost, X_test_boost, y_test_boost)
        
    # Add model name and Gini coefficient to the DataFrame
    if gini_prob is not None:
        gini_df = pd.concat([gini_df, pd.DataFrame({'Model': [model_name], 'Gini_prob': [gini_prob]})], ignore_index=True)

# Sort the DataFrame by Gini coefficient in descending order
gini_df_sorted = gini_df.sort_values(by='Gini_prob', ascending=False)

Model Performance for XGBoost
Gini prob is 94.64229910714286
              precision    recall  f1-score   support

           0       0.94      0.97      0.95      7040
           1       0.87      0.79      0.83      1960

    accuracy                           0.93      9000
   macro avg       0.91      0.88      0.89      9000
weighted avg       0.93      0.93      0.93      9000

[[6803  237]
 [ 405 1555]]
[LightGBM] [Info] Number of positive: 4676, number of negative: 16324
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001183 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1233
[LightGBM] [Info] Number of data points in the train set: 21000, number of used features: 23
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.222667 -> initscore=-1.250193
[LightGBM] [Info] Start training from score -1.250193
Model Performance for LightG

In [131]:
# Display the sorted DataFrame containing model names and corresponding Gini coefficients
gini_df_sorted

,Model,Gini_prob
2,CatBoost,0.949452
1,LightGBM,0.949256
0,XGBoost,0.946423
3,CatBoost_Custom,0.939026
4,LogisticRegression,0.924391
5,RandomForest,0.917801
6,SVC,0.916485


# OPTUNA

In [142]:
def best_params_for_model(trial):
    # Define the parameter space to be explored by Optuna
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),  # Number of boosting rounds
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),  # Learning rate for boosting
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Maximum depth of each tree
        'num_leaves': trial.suggest_int('num_leaves', 10, 100)  # Maximum number of leaves in a tree
    }

    # Initialize a LightGBM classifier with the suggested parameters
    lgb_clf = LGBMClassifier(**param)

    # Evaluate the classifier using cross-validation with ROC AUC as the scoring metric
    auc = cross_val_score(lgb_clf, X_train_boost, y_train_boost, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

# Create a study object for optimization with Optuna, maximizing AUC
study = optuna.create_study(direction='maximize')

# Optimize the parameters using the defined function and a limited number of trials
study.optimize(best_params_for_model, n_trials=1)

# Print the results of the best trial found by Optuna
print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

# Instantiate the best LightGBM model with the optimal parameters
best_lgb_model = LGBMClassifier(**best_params)

[I 2024-05-19 22:30:22,498] A new study created in memory with name: no-name-325b0564-63d4-44d7-9543-e53c46692e10
[I 2024-05-19 22:30:36,044] Trial 0 finished with value: 0.9640212310729791 and parameters: {'n_estimators': 888, 'learning_rate': 0.11252774409904197, 'max_depth': 9, 'num_leaves': 68}. Best is trial 0 with value: 0.9640212310729791.


Best trial:
  Value: 0.964
  Params:  {'n_estimators': 888, 'learning_rate': 0.11252774409904197, 'max_depth': 9, 'num_leaves': 68}


In [143]:
def best_params_for_model(trial):
    # Define the parameter space to be explored by Optuna
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),  # Number of boosting rounds
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),  # Learning rate for boosting
        'max_depth': trial.suggest_int('max_depth', 3, 10),  # Maximum depth of each tree
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),  # Fraction of the training data used to train each tree
        # Using a fraction of the data can help to prevent overfitting and improve generalization
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),  # Subsample ratio of columns when constructing each tree
        'gamma': trial.suggest_int('gamma', 0, 5)
        # Increasing the gamma value can help to prevent overfitting
        # Should be used only when you are using high depth
    }
    
    # Initialize an XGBoost classifier with the suggested parameters
    xgb_clf = XGBClassifier(**param)

    # Evaluate the classifier using cross-validation with ROC AUC as the scoring metric
    auc = cross_val_score(xgb_clf, X_train_boost, y_train_boost, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

# Create a study object for optimization with Optuna, maximizing AUC
study = optuna.create_study(direction='maximize')

# Optimize the parameters using the defined function and a limited number of trials
study.optimize(best_params_for_model, n_trials=10)

# Print the results of the best trial found by Optuna
print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

# Instantiate the best XGBoost model with the optimal parameters
best_xgb_model = XGBClassifier(**best_params)

[I 2024-05-19 22:31:34,305] A new study created in memory with name: no-name-9e8a83a1-b866-4f9e-9963-743c1addf205
[I 2024-05-19 22:31:35,887] Trial 0 finished with value: 0.9690890766101988 and parameters: {'n_estimators': 25, 'learning_rate': 0.3990232127652329, 'max_depth': 8, 'subsample': 0.9959051716790508, 'colsample_bytree': 0.7736954329821081, 'gamma': 3}. Best is trial 0 with value: 0.9690890766101988.
[I 2024-05-19 22:31:38,795] Trial 1 finished with value: 0.9705134556344727 and parameters: {'n_estimators': 620, 'learning_rate': 0.024682252086997595, 'max_depth': 4, 'subsample': 0.6655547619644708, 'colsample_bytree': 0.9895956252039916, 'gamma': 4}. Best is trial 1 with value: 0.9705134556344727.
[I 2024-05-19 22:31:41,056] Trial 2 finished with value: 0.9707115694360073 and parameters: {'n_estimators': 560, 'learning_rate': 0.18947474762248448, 'max_depth': 3, 'subsample': 0.5842650079260303, 'colsample_bytree': 0.8598009125560981, 'gamma': 4}. Best is trial 2 with value: 0

Best trial:
  Value: 0.971
  Params:  {'n_estimators': 981, 'learning_rate': 0.014694172188857433, 'max_depth': 9, 'subsample': 0.8201368627152424, 'colsample_bytree': 0.8349484649073864, 'gamma': 2}


In [144]:
def best_params_for_model(trial):
    # Define the parameter space to be explored by Optuna
    param = {
        'iterations': trial.suggest_int('iterations', 100, 1000),  # Number of boosting iterations
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),  # Learning rate for boosting
        'depth': trial.suggest_int('depth', 3, 10),  # Depth of the trees
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 0.1, 10),  # Regularization term that prevents overfitting
        # 'cat_features': [],  # Handle categorical features separately
        'loss_function': trial.suggest_categorical('loss_function', ['Logloss'])  # Loss function used for training
        # For regression tasks, use ‘RMSE,’ while for classification, use ‘Logloss’.
    }

    # Initialize a CatBoost classifier with the suggested parameters
    cb_clf = CatBoostClassifier(**param)

    # Evaluate the classifier using cross-validation with ROC AUC as the scoring metric
    auc = cross_val_score(cb_clf, X_train_boost, y_train_boost, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

# Create a study object for optimization with Optuna, maximizing AUC
study = optuna.create_study(direction='maximize')

# Optimize the parameters using the defined function and a limited number of trials
study.optimize(best_params_for_model, n_trials=1)

# Print the results of the best trial found by Optuna
print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

# Instantiate the best CatBoost model with the optimal parameters
best_cb_model = CatBoostClassifier(**best_params)

[I 2024-05-19 22:32:18,645] A new study created in memory with name: no-name-c10e8407-0744-4ac4-ada2-cb537f5aade8
[I 2024-05-19 22:32:40,347] Trial 0 finished with value: 0.9679453665489163 and parameters: {'iterations': 236, 'learning_rate': 0.010419416701655259, 'depth': 10, 'l2_leaf_reg': 0.11576912586711254, 'loss_function': 'Logloss'}. Best is trial 0 with value: 0.9679453665489163.


Best trial:
  Value: 0.968
  Params:  {'iterations': 236, 'learning_rate': 0.010419416701655259, 'depth': 10, 'l2_leaf_reg': 0.11576912586711254, 'loss_function': 'Logloss'}


In [145]:
def best_params_for_model(trial):
    # Define the parameter space to be explored by Optuna
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 10, 1000),  # Number of trees in the forest
        'max_depth': trial.suggest_int('max_depth', 3, 20),  # Maximum depth of each tree
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),  # Minimum number of samples required to split an internal node
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),  # Minimum number of samples required to be at a leaf node
        'max_features': trial.suggest_uniform('max_features', 0.1, 1.0),  # Number of features to consider when looking for the best split
        'bootstrap': trial.suggest_categorical('bootstrap', [True, False])  # Whether bootstrap samples are used when building trees
    }

    # Initialize a Random Forest classifier with the suggested parameters
    rf_clf = RandomForestClassifier(**param, n_jobs=-1)
    
    # Evaluate the classifier using cross-validation with ROC AUC as the scoring metric
    auc = cross_val_score(rf_clf, X_train_boost, y_train_boost, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

# Create a study object for optimization with Optuna, maximizing AUC
study = optuna.create_study(direction='maximize')

# Optimize the parameters using the defined function and a limited number of trials
study.optimize(best_params_for_model, n_trials=10)

# Print the results of the best trial found by Optuna
print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

# Instantiate the best Random Forest model with the optimal parameters
best_rf_model = RandomForestClassifier(**best_params, n_jobs=-1)

# Fit the model on the training data
best_rf_model.fit(X_train_boost, y_train_boost)

[I 2024-05-19 22:33:57,724] A new study created in memory with name: no-name-fcabdf88-f65e-464c-a6e6-6870f43ed12d
[I 2024-05-19 22:34:16,211] Trial 0 finished with value: 0.950213297985541 and parameters: {'n_estimators': 233, 'max_depth': 13, 'min_samples_split': 16, 'min_samples_leaf': 5, 'max_features': 0.533122315250195, 'bootstrap': False}. Best is trial 0 with value: 0.950213297985541.
[I 2024-05-19 22:35:24,675] Trial 1 finished with value: 0.9356963338661989 and parameters: {'n_estimators': 841, 'max_depth': 10, 'min_samples_split': 20, 'min_samples_leaf': 20, 'max_features': 0.7263116802686009, 'bootstrap': False}. Best is trial 0 with value: 0.950213297985541.
[I 2024-05-19 22:35:32,018] Trial 2 finished with value: 0.928084532261915 and parameters: {'n_estimators': 318, 'max_depth': 6, 'min_samples_split': 10, 'min_samples_leaf': 8, 'max_features': 0.3341865100592114, 'bootstrap': True}. Best is trial 0 with value: 0.950213297985541.
[I 2024-05-19 22:35:39,221] Trial 3 finis

Best trial:
  Value: 0.952
  Params:  {'n_estimators': 681, 'max_depth': 15, 'min_samples_split': 9, 'min_samples_leaf': 5, 'max_features': 0.5851223179094796, 'bootstrap': True}


RandomForestClassifier(max_depth=15, max_features=0.5851223179094796,
                       min_samples_leaf=5, min_samples_split=9,
                       n_estimators=681, n_jobs=-1)

In [147]:
def best_params_for_model(trial):
    # Define the parameter space to be explored by Optuna
    param = {
        'C': trial.suggest_loguniform('C', 1e-5, 1e2),  # Regularization parameter
        'kernel': trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid']),  # Kernel type
        # Degree of the polynomial kernel (used only if 'poly' kernel is selected)
        'degree': trial.suggest_int('degree', 2, 5) if 'poly' in trial.params['kernel'] else 3,
        'gamma': trial.suggest_categorical('gamma', ['scale', 'auto']),  # Kernel coefficient
        # Independent term in kernel function (used for 'poly' and 'sigmoid' kernels)
        'coef0': trial.suggest_uniform('coef0', 0.0, 1.0) if 'poly' in trial.params['kernel'] or 'sigmoid' in trial.params['kernel'] else 0.0
    }

    # Initialize a Support Vector Classifier with the suggested parameters
    svc_clf = SVC(**param)

    # Create a pipeline with standard scaler and SVC
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', svc_clf)
    ])
    
    # Evaluate the pipeline using cross-validation with ROC AUC as the scoring metric
    auc = cross_val_score(pipeline, X_train_log, y_train_log, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    return auc

# Create a study object for optimization with Optuna, maximizing AUC
study = optuna.create_study(direction='maximize')

# Optimize the parameters using the defined function and a limited number of trials
study.optimize(best_params_for_model, n_trials=10)

# Print the results of the best trial found by Optuna
print('Best trial:')
best_params = study.best_params
print('  Value: {:.3f}'.format(study.best_value))
print('  Params: ', best_params)

# Instantiate the best Support Vector Classifier with the optimal parameters
svc_clf = SVC(**best_params)

# Create a pipeline with standard scaler and the best SVC
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', svc_clf)
])

# Fit the pipeline on the training data
pipeline.fit(X_train_boost, y_train_boost)

[I 2024-05-19 22:40:51,055] A new study created in memory with name: no-name-68c20498-abe2-4396-a309-eba720bc3767
[I 2024-05-19 22:41:00,677] Trial 0 finished with value: 0.9623133498370501 and parameters: {'C': 0.8082945883716343, 'kernel': 'linear', 'gamma': 'auto'}. Best is trial 0 with value: 0.9623133498370501.
[I 2024-05-19 22:41:07,122] Trial 1 finished with value: 0.961418783547547 and parameters: {'C': 0.12470791689676829, 'kernel': 'poly', 'degree': 2, 'gamma': 'auto', 'coef0': 0.1529357490540587}. Best is trial 0 with value: 0.9623133498370501.
[I 2024-05-19 22:41:30,086] Trial 2 finished with value: 0.9294867307630819 and parameters: {'C': 0.0029805875105791916, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.9623133498370501.
[I 2024-05-19 22:41:38,978] Trial 3 finished with value: 0.9623870988339105 and parameters: {'C': 5.250716501814062, 'kernel': 'poly', 'degree': 2, 'gamma': 'auto', 'coef0': 0.24168910873535177}. Best is trial 3 with value: 0.9623870

Best trial:
  Value: 0.963
  Params:  {'C': 1.0318665922630539, 'kernel': 'poly', 'degree': 2, 'gamma': 'auto', 'coef0': 0.6295222414115217}


Pipeline(steps=[('scaler', StandardScaler()),
                ('svc',
                 SVC(C=1.0318665922630539, coef0=0.6295222414115217, degree=2,
                     gamma='auto', kernel='poly'))])

In [150]:
# List to store optimized models along with their names
models_optimized = []

# Extend the list with tuples containing model names and corresponding optimized models
models_optimized.extend([
    ('XGBoost Optuna', best_xgb_model),         # XGBoost model optimized with Optuna
    ('LightGBM Optuna', best_lgb_model),        # LightGBM model optimized with Optuna
    ('CatBoost Optuna', best_cb_model),         # CatBoost model optimized with Optuna
    ('RandomForest Optuna', best_rf_model),     # Random Forest model optimized with Optuna
    ('SVC Optuna', pipeline)                    # Support Vector Classifier pipeline optimized with Optuna
])

In [153]:
# Create an empty DataFrame to store Gini probabilities for each optimized model
gini_df_optuna = pd.DataFrame(columns=['Model', 'Gini_prob'])

# Iterate over each model and its corresponding name in the list of optimized models
for model_name, model in models_optimized:
    # Cross-validate the model and calculate the AUC
    if 'SVC' in model_name:
        auc = cross_val_score(model, X_train_log, y_train_log, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    else:
        auc = cross_val_score(model, X_train_boost, y_train_boost, cv=3, scoring='roc_auc', n_jobs=-1).mean()
    
    # Calculate the Gini probability from the AUC
    gini_prob = 2 * auc - 1
    
    # Check if the Gini probability is not None
    if gini_prob is not None:
        # Append the model name and its corresponding Gini probability to the DataFrame
        gini_df_optuna = pd.concat([gini_df_optuna, pd.DataFrame({'Model': [model_name], 'Gini_prob': [gini_prob]})], ignore_index=True)

# Sort the DataFrame by Gini probability in descending order
gini_df_sorted_optuna = gini_df_optuna.sort_values(by='Gini_prob', ascending=False)

# Print the sorted DataFrame
print(gini_df_sorted_optuna)

                 Model  Gini_prob
0       XGBoost Optuna   0.942898
2      CatBoost Optuna   0.935891
1      LightGBM Optuna   0.928045
4           SVC Optuna   0.926052
3  RandomForest Optuna   0.904205


In [154]:
# Concatenate the sorted Gini DataFrames from both regular optimization and Optuna optimization
final_review = pd.concat([gini_df_sorted, gini_df_sorted_optuna], axis=0)

# Sort the concatenated DataFrame by Gini probability in descending order
final_review_sorted = final_review.sort_values(by='Gini_prob', ascending=False)

# Reset the index of the sorted DataFrame
final_review_sorted.reset_index(drop=True, inplace=True)

# Print the final sorted DataFrame
final_review_sorted

,Model,Gini_prob
0,CatBoost,0.949452
1,LightGBM,0.949256
2,XGBoost,0.946423
3,XGBoost Optuna,0.942898
4,CatBoost_Custom,0.939026
5,CatBoost Optuna,0.935891
6,LightGBM Optuna,0.928045
7,SVC Optuna,0.926052
8,LogisticRegression,0.924391
9,RandomForest,0.917801
